# Ejercicio 11 : Asistente RAG Conversacional

## Objetivo de la práctica

Construir un asistente que:

1. Recibe una pregunta del usuario
2. Recupera texto relevante de un corpus (ej. libro de Baeza-Yates)
3. Genera una respuesta basada en los documentos encontrados
4. Mantiene el historial de conversación


## Parte 0: Librerías necesarias
- openai
- faiss-cpu
- sentence-transformers

In [1]:
import openai
import faiss
import numpy as np
import PyPDF2
import re
from sentence_transformers import SentenceTransformer
from typing import List, Dict, Tuple
import json
import os
from datetime import datetime
import pandas as pd

In [39]:
# Inicializar el modelo de embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

## Parte 1: Carga del corpus

Aquí se debe cargar el corpus con los documentos en PDF.

- Libro de Stanford
- Libro BM25
- Paper: Marcia Bates (1989). The design of browsing and berrypicking techniques for the online search interface

In [3]:
import fitz  # PyMuPDF

doc_bates = fitz.open(r"D:\Universidad\8 - Octavo\Recuperacion de la informacion\Tareas\Tarea11\corpus_bates.pdf")
doc_bm25 = fitz.open(r"D:\Universidad\8 - Octavo\Recuperacion de la informacion\Tareas\Tarea11\corpus_bm25.pdf")
doc_stand = fitz.open(r"D:\Universidad\8 - Octavo\Recuperacion de la informacion\Tareas\Tarea11\corpus_standfoard.pdf")

## Parte 2: Procesamiento del Corpus

Aquí se debe obtener el corpus procesado. El corpus estará formado por documentos que corresponden a las secciones (o subsecciones) de los libros. Cada documento debe indicar a qué libro corresponde, así como las páginas en las que está dentro de ese libro.

Recuerden que los documentos procesados no deben contener docos o caracteres ajenos al tema del que tratan.  

In [4]:
#Saber si el PDF tiene una tabla de contenidos estructurada
def check_pdf_toc(doc):
    toc = doc.get_toc()
    if toc:
        print(f"El PDF tiene una tabla de contenidos estructurada con {len(toc)} entradas.")
    else:
        print("El PDF no tiene tabla de contenidos estructurada.") 

In [5]:
#check_pdf_toc(doc_bates)
#check_pdf_toc(doc_stand)
#check_pdf_toc(doc_bm25)

In [6]:
#Funcion para extraer texto de un documento PDF
def extract_text(doc):
    data = []
    for i, page in enumerate(doc):
        text = page.get_text().encode('utf-8')
        if text.strip():
            data.append({
                "page": i + 1,
                "raw": str(text).strip()
            })
    return data

In [7]:
#Extraer texto de los documentos
pages_doc_bates = extract_text(doc_bates)
pages_doc_bm25 = extract_text(doc_bm25)
pages_doc_stand  = extract_text(doc_stand)

In [8]:
# Crear DataFrames para cada documento
df_bates = pd.DataFrame(pages_doc_bates)
df_bm25 = pd.DataFrame(pages_doc_bm25)
df_stand = pd.DataFrame(pages_doc_stand)

In [ ]:
#Mostrar los DataFrames
#df_bates
#df_bm25 
#df_stand

In [9]:
def limpiar_texto(text):
    """Limpia el texto extraído de caracteres no deseados"""
    # Remover el prefijo b' y el sufijo '
    if text.startswith("b'") and text.endswith("'"):
        text = text[2:-1]
    
    # Reemplazar secuencias de escape
    text = text.replace('\\n', '\n').replace('\\t', '\t').replace('\\r', '\r')
    
    # Remover caracteres especiales pero mantener puntuación básica
    text = re.sub(r'[^\w\s\.\,\;\:\!\?\-\(\)\[\]\"\'\/\%\&\*\+\=\<\>\@\#\$]', '', text)
    
    # Normalizar espacios en blanco
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

In [11]:
def divide_by_paragraphs(text, min_length=200, max_length=1500):
    """Divide el texto en párrafos cuando no se detectan secciones"""
    paragraphs = text.split('\n\n')
    sections = []
    current_section = ""
    section_num = 1
    
    for paragraph in paragraphs:
        paragraph = paragraph.strip()
        if not paragraph:
            continue
        
        # Si agregar este párrafo excede el límite máximo, crear nueva sección
        if len(current_section) + len(paragraph) > max_length and len(current_section) > min_length:
            sections.append({
                'title': f'Sección {section_num}',
                'content': current_section.strip(),
                'start_pos': 0,  # Aproximado
                'end_pos': 0     # Aproximado
            })
            current_section = paragraph
            section_num += 1
        else:
            current_section += "\n\n" + paragraph if current_section else paragraph
    
    # Agregar la última sección
    if current_section.strip() and len(current_section) > min_length:
        sections.append({
            'title': f'Sección {section_num}',
            'content': current_section.strip(),
            'start_pos': 0,
            'end_pos': 0
        })
    
    return sections

In [12]:
def dividir_secciones(text):
    # Patrones para detectar títulos de secciones
    section_patterns = [
        r'\n\s*\d+\.\s*[A-Z][^\n]*\n',           # 1. Título
        r'\n\s*\d+\.\d+\s*[A-Z][^\n]*\n',       # 1.1 Subtítulo
        r'\n\s*\d+\.\d+\.\d+\s*[A-Z][^\n]*\n', # 1.1.1 Sub-subtítulo
        r'\n\s*[A-Z][A-Z\s]{2,}\n',              # TÍTULOS EN MAYÚSCULAS
        r'\n\s*[A-Z][^.\n]*\n\s*\n',            # Título seguido de línea vacía
        r'\n\s*Chapter\s+\d+[^\n]*\n',          # Chapter X
        r'\n\s*Section\s+\d+[^\n]*\n',          # Section X
        r'\n\s*Abstract\s*\n',                   # Abstract
        r'\n\s*Introduction\s*\n',               # Introduction
        r'\n\s*Conclusion\s*\n',                 # Conclusion
        r'\n\s*References\s*\n',                 # References
        r'\n\s*Bibliography\s*\n',               # Bibliography
    ]
    
    sections = []
    current_pos = 0
    
    # Buscar todos los patrones
    all_matches = []
    for pattern in section_patterns:
        matches = list(re.finditer(pattern, text, re.IGNORECASE))
        for match in matches:
            all_matches.append((match.start(), match.end(), match.group().strip()))
    
    # Ordenar por posición
    all_matches.sort(key=lambda x: x[0])
    
    # Si no se encuentran secciones, dividir por párrafos largos
    if not all_matches:
        return divide_by_paragraphs(text)
    
    # Dividir el texto en secciones
    for i, (start, end, title) in enumerate(all_matches):
        # Contenido de la sección anterior
        if i == 0:
            # Contenido antes de la primera sección
            if start > 0:
                content = text[current_pos:start].strip()
                if len(content) > 100:
                    sections.append({
                        'title': 'Introducción/Preámbulo',
                        'content': content,
                        'start_pos': current_pos,
                        'end_pos': start
                    })
        
        # Encontrar el final de esta sección
        if i < len(all_matches) - 1:
            section_end = all_matches[i + 1][0]
        else:
            section_end = len(text)
        
        # Contenido de la sección actual
        section_content = text[end:section_end].strip()
        if len(section_content) > 100:
            sections.append({
                'title': title,
                'content': section_content,
                'start_pos': end,
                'end_pos': section_end
            })
    
    return sections

In [13]:
def estimate_page_range(text_position, total_text_length, total_pages):
    """Estima el rango de páginas basado en la posición del texto"""
    if total_text_length == 0:
        return 1, 1
    
    # Calcular página aproximada
    page_ratio = text_position / total_text_length
    estimated_page = max(1, int(page_ratio * total_pages))
    
    return estimated_page, estimated_page

In [14]:
def process_documents_with_sections(df_list, doc_names):
    """Procesa los DataFrames y divide el contenido en secciones"""
    processed_docs = []
    
    for df, doc_name in zip(df_list, doc_names):
        print(f"Procesando {doc_name}...")
        
        # Concatenar todo el texto del documento
        full_text = ""
        page_texts = []
        
        for index, row in df.iterrows():
            clean_content = limpiar_texto(row['raw'])
            if len(clean_content) > 50:  # Filtrar páginas casi vacías
                page_texts.append((row['page'], clean_content))
                full_text += f"\n\n--- PÁGINA {row['page']} ---\n\n" + clean_content
        
        print(f"Texto completo: {len(full_text)} caracteres")
        
        # Detectar secciones en el texto completo
        sections = dividir_secciones(full_text)
        print(f"Secciones detectadas: {len(sections)}")
        
        # Crear documentos por sección
        for i, section in enumerate(sections):
            # Estimar páginas basándose en la posición del texto
            start_page, end_page = estimate_page_range(
                section['start_pos'], 
                len(full_text), 
                len(page_texts)
            )
            
            # Buscar páginas más precisas basándose en el contenido
            actual_pages = []
            section_content = section['content']
            
            for page_num, page_content in page_texts:
                # Si hay overlap significativo entre el contenido de la sección y la página
                overlap = len(set(section_content.split()) & set(page_content.split()))
                if overlap > 10:  # Umbral de overlap
                    actual_pages.append(page_num)
            
            if actual_pages:
                start_page, end_page = min(actual_pages), max(actual_pages)
            
            doc = {
                'id': f"{doc_name}_section_{i+1}",
                'content': section['content'],
                'source': doc_name,
                'section_title': section['title'],
                'section_number': i + 1,
                'page_start': start_page,
                'page_end': end_page,
                'length': len(section['content']),
                'pages': actual_pages if actual_pages else [start_page]
            }
            processed_docs.append(doc)
        
        print(f"✓ {doc_name}: {len([d for d in processed_docs if d['source'] == doc_name])} secciones procesadas")
    
    return processed_docs

In [15]:
document_names = ['Bates_1989', 'BM25_Book', 'Stanford_Book']
processed_documents = process_documents_with_sections([df_bates, df_bm25, df_stand], document_names)

Procesando Bates_1989...
Texto completo: 54296 caracteres
Secciones detectadas: 18
✓ Bates_1989: 18 secciones procesadas
Procesando BM25_Book...
Texto completo: 105009 caracteres
Secciones detectadas: 22
✓ BM25_Book: 22 secciones procesadas
Procesando Stanford_Book...
Texto completo: 1361638 caracteres
Secciones detectadas: 3
✓ Stanford_Book: 3 secciones procesadas


## Parte 3: Cálculo de Embeddings e Indexación en base de datos vectorial

Aquí, una vez que se ha calculado el embedding de cada documento, se deberá indexar este embedding en una base de datos vectorial como FAISS, ChromaDB o Pinecone

In [16]:
def create_embeddings(documents):
    # Extraer el contenido de texto
    texts = [doc['content'] for doc in documents]
    
    # Calcular embeddings usando sentence-transformers
    embeddings = embedding_model.encode(texts, batch_size=32)
    
    return embeddings

In [17]:
def create_faiss_index(embeddings):
    """Crea índice FAISS para búsqueda vectorial"""
    dimension = embeddings.shape[1]
    
    # Crear índice FAISS usando Inner Product (para cosine similarity)
    index = faiss.IndexFlatIP(dimension)
    
    # Normalizar embeddings para cosine similarity
    embeddings_normalized = embeddings.copy()
    faiss.normalize_L2(embeddings_normalized)
    
    # Agregar embeddings al índice
    index.add(embeddings_normalized.astype('float32'))
    
    print(f"✓ Índice FAISS creado con {index.ntotal} documentos")
    return index, embeddings_normalized


In [18]:
document_embeddings = create_embeddings(processed_documents)

In [19]:
faiss_index, normalized_embeddings = create_faiss_index(document_embeddings)

✓ Índice FAISS creado con 43 documentos


## Parte 4: Búsqueda y obtención del contexto

En esta parte debemos definir una _query_ y buscar los documentos que más se relacionan con ella.

Estos documentos formarán el contexto que vamos a entregar al LLM.

In [21]:
def search_similar_documents(query: str, index, embeddings, documents, top_k=3):
    """
    Busca documentos similares a la query usando FAISS
    """
    # Calcular embedding de la query
    query_embedding = embedding_model.encode([query])
    
    # Normalizar el embedding de la query
    faiss.normalize_L2(query_embedding)
    
    # Buscar documentos similares
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    # Preparar resultados
    results = []
    for score, idx in zip(scores[0], indices[0]):
        doc = documents[idx]
        result = {
            'score': float(score),
            'document': doc,
            'content': doc['content'],
            'source': f"{doc['source']}, páginas {doc['pages']}"
        }
        results.append(result)
    
    return results

In [22]:
def prepare_context(query: str, results: List[Dict], max_length: int = 4000) -> str:
    context_parts = []
    total_length = 0
    
    # Agregar la query
    context_parts.append(f"Pregunta: {query}\n")
    context_parts.append("\nContexto relevante de los documentos:\n")
    
    # Ordenar resultados por score de similitud
    sorted_results = sorted(results, key=lambda x: x['score'], reverse=True)
    
    # Agregar cada documento encontrado
    for i, result in enumerate(sorted_results, 1):
        doc = result['document']
        
        # Crear encabezado del documento
        header = f"\nFuente {i} ({result['score']:.3f}): {doc['source']}"
        header += f"\nSección: {doc['section_title']}"
        header += f"\nPáginas: {', '.join(map(str, doc['pages']))}"
        
        # Calcular longitud del contenido
        content = f"Contenido:\n{result['content']}\n"
        section_length = len(header) + len(content)
        
        # Verificar si agregar esta sección excedería el límite
        if total_length + section_length > max_length:
            content = content[:max_length - total_length - len(header) - 20] + "..."
        
        context_parts.append(header)
        context_parts.append(content)
        
        total_length += len(header) + len(content)
        
        # Detener si alcanzamos el límite de longitud
        if total_length >= max_length:
            context_parts.append("\n[Contexto truncado por longitud máxima]")
            break
    
    return "\n".join(context_parts)

In [23]:
# Función para ejecutar una búsqueda completa
def run_search(query: str, index, embeddings, documents, top_k=3):
    """
    Ejecuta una búsqueda completa y prepara el contexto
    """
    # Buscar documentos similares
    search_results = search_similar_documents(query, index, embeddings, documents, top_k)
    
    # Preparar contexto
    context = prepare_context(query, search_results)
    
    return context, search_results



## Parte 5: Generación de Respuesta

Aquí, entregamos el contexto al LLM, y él nos responde a la _query_ con una explicación en lenguaje natural.

## Parte 5: Generación de respuesta con OpenAI

In [ ]:
from openai import OpenAI

client = OpenAI()

In [30]:
def generate_answer(context: str, model="gpt-3.5-turbo", temperature=0.2) -> str:
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "Eres un asistente experto en recuperación de información."},
                {"role": "user", "content": context}
            ],
            temperature=temperature
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error en la generación de respuesta: {e}"


## Parte 6: Bucle conversacional (chat)

In [ ]:
conversation_history = []

def ask_question(user_input: str, model="gpt-3.5-turbo", temperature=0.2):

    # Buscar documentos relevantes y preparar contexto
    context, _ = run_search(user_input, faiss_index, normalized_embeddings, processed_documents)

    # Generar respuesta con OpenAI
    respuesta = generate_answer(context, model=model, temperature=temperature)

    # Agregar al historial
    conversation_history.append({
        "usuario": user_input,
        "asistente": respuesta
    })

    # Mostrar respuesta
    print(f"Asistente: {respuesta}\n")
    return respuesta


In [37]:
ask_question("¿Quien propuso BM25?")


Asistente: El modelo BM25 fue propuesto por Stephen E. Robertson, Karen Spärck Jones y Steve Walker en 1995.



'El modelo BM25 fue propuesto por Stephen E. Robertson, Karen Spärck Jones y Steve Walker en 1995.'

In [38]:
for t in conversation_history:
    print("Usuario:", t["usuario"])
    print("Asistente:", t["asistente"])
    print("-" * 50)

Usuario: ¿Qué es BM25?
Asistente: Error en la generación de respuesta: No module named 'openai.resources'
--------------------------------------------------
Usuario: ¿Qué es BM25?
Asistente: BM25 (Best Matching 25) es un modelo de ponderación de términos utilizado en la recuperación de información. Se utiliza para calcular el peso de los términos en un documento en función de su frecuencia y de la longitud del documento. El modelo BM25 tiene parámetros internos como b y k1 que deben ser ajustados para adaptarse a los datos y mejorar la precisión de la recuperación de información.
--------------------------------------------------
Usuario: ¿Quien propuso BM25?
Asistente: El modelo BM25 fue propuesto por Stephen E. Robertson, Karen Spärck Jones y Steve Walker en 1995.
--------------------------------------------------
